# Data Science and Visualization (RUC F2024)

## Lecture 2: Exploratory Data Analysis (EDA)

# Missing values

In [1]:
import numpy as np
import pandas as pd

# 1. Types of Missing Values

## (1) np.nan, np.NaN, np.NAN

1. If a column is numeric and you have a missing value, that value will be a NaN. NaN stands for Not a Number.

2. NaNs are always floats. So if you have an integer column and it has a NaN added to it, the column is upcasted to become a float column. 

3. NaN doesn't equal NaN :)

4. After a dataset is loaded, the default missing values all become NaN. Sometimes, due to the missing values, a dimension's dtype seems to be weird. For example, a dimension with float values (and NaNs) will be shown as *object* dtype, though NaN is float.

In [2]:
type(np.nan)

float

In [3]:
np.nan == np.nan

False

NB: **np.nan**, **np.NaN**, and **np.NAN** are the same:

In [4]:
np.nan

nan

In [5]:
np.NaN

nan

In [6]:
np.NAN

nan

These variables are not defined in np:

In [7]:
#np.Nan
#np.nAn
#np.naN
#np.NAn
#np.nAN

## (2) pd.NaT

1. NaT stands for Not a Time.
2. Missing value for a DateTime column.
3. NaT doesn't equal NaT.

In [8]:
pd.NaT

NaT

In [9]:
pd.NaT == pd.NaT

False

## (3) Python's None

1. None defines a null value, or no value at all. It’s not the same as 0, False (or Boolean), or an empty string. It’s a data type of its own (NoneType) and only None can be None.
2. None equals None.
3. When a dimension has other dtypes, None is often converted to np.NaN. So we seldom see None, unless we generate it on purpose.

In [10]:
None == None

True

In [11]:
s1 = pd.Series([1, None])
s1[1]

nan

In [12]:
type(s1[1])

numpy.float64

In [13]:
s1

0    1.0
1    NaN
dtype: float64

We can force a missing value to be NoneType.

In [14]:
se = pd.Series([1, None], dtype='O')
se

0       1
1    None
dtype: object

In [15]:
type(se[1])

NoneType

In [16]:
type(se[0])

int

In [17]:
se2 = pd.Series(['Alex', None], dtype='O')
se2

0    Alex
1    None
dtype: object

In [18]:
type(se2[0])

str

In [19]:
type(se2[1])

NoneType

# 2. When Different Types Meet (optional)

When different missing value types meet in the same dimension, the situation becomes complicated. Type conversion (type cast) will be done automatically according to some implicit rules. This part is optional---you don't have to read through it.

When **np.nan**, **pd.NaT** and **None** are on the same column:

In [20]:
df = pd.DataFrame([[np.nan, np.nan, np.nan, None, 3], 
                   [4,      5,      None,   None, None], 
                   [7,      pd.NaT, pd.NaT, pd.NaT, pd.NaT]]
                 )
                    #, columns=['A', 'B', 'C', 'D'])
df

,0,1,2,3,4
0,NaN,NaN,NaT,NaT,3
1,4.0,5,NaT,NaT,None
2,7.0,NaT,NaT,NaT,NaT


Above, pay attention to columns C and D. If all values are missing on a column, and pd.NaT is present, all missing values will be converted to pd.NaT as it is the strongest (most specific) type. If there is at least one concrete value, None will be kept.

We write df to a disk file and reload it to the memory. All missing values are converted to NaN!

In [21]:
df.to_csv('C:/Data/missing_values_1.csv')

In [22]:
df_1 = pd.read_csv('C:/Data/missing_values_1.csv', index_col=[0])
df_1

,0,1,2,3,4
0,NaN,NaN,NaN,NaN,3.0
1,4.0,5.0,NaN,NaN,NaN
2,7.0,NaN,NaN,NaN,NaN


In [23]:
df = pd.DataFrame([[0,    'RUC',    2,    3],
                   [None, 5,    None, pd.NaT],
                   [8,    None, 10,   None],
                   [11,   12,   13,   pd.NaT]],
                  columns=list('ABCD'))
df

,A,B,C,D
0,0.0,RUC,2.0,3
1,NaN,5,NaN,NaT
2,8.0,None,10.0,None
3,11.0,12,13.0,NaT


More examples about the dtype of pd.NaT:

In [24]:
pd.Series([np.NaN, pd.NaT, None])

0   NaT
1   NaT
2   NaT
dtype: datetime64[ns]

Pandas created the Series as a DateTime dtype because pd.NaT indicates a strong type. We can cast it to an object dtype if you like.

In [25]:
pd.Series([np.NaN, pd.NaT, None]).astype('object')

0    NaT
1    NaT
2    NaT
dtype: object

But we can't cast it to a numeric dtype.

In [26]:
pd.Series([np.NaN, pd.NaT, None]).astype('float')

TypeError: Cannot cast DatetimeArray to dtype float64

# 3. Missing Value Judgement/Detection

To judge if a DataFrame contains missing values, we have two sets of functions that all return True or False.
* **isnull()** and **notnull()**
* **isna()** and **notna()**

They all can be applied to the whole DataFrame, one column, one row, or a slice.

In the following example, pay attention to that how None is interpreted differently for a dimesion with strings and one with int values.

In [27]:
df = pd.DataFrame([['a1', 'b1', 1, 5],
                   ['a2', None, 2, 6],
                   ['a3', 'b3', None, 9],
                   ['a4', 'b4', 4, pd.NaT]],
                 columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,a1,b1,1.0,5
1,a2,None,2.0,6
2,a3,b3,NaN,9
3,a4,b4,4.0,NaT


## (1) For the whole DataFrame

A DataFrame with boolean values is returned.

In [28]:
df.isnull()

,A,B,C,D
0,False,False,False,False
1,False,True,False,False
2,False,False,True,False
3,False,False,False,True


In [29]:
df.notnull()

,A,B,C,D
0,True,True,True,True
1,True,False,True,True
2,True,True,False,True
3,True,True,True,False


In [30]:
df.isna()

,A,B,C,D
0,False,False,False,False
1,False,True,False,False
2,False,False,True,False
3,False,False,False,True


In [31]:
df.notna()

,A,B,C,D
0,True,True,True,True
1,True,False,True,True
2,True,True,False,True
3,True,True,True,False


## (2) For one column

In [32]:
df['C'].notnull()

0     True
1     True
2    False
3     True
Name: C, dtype: bool

In [33]:
df['C'].isna()

0    False
1    False
2     True
3    False
Name: C, dtype: bool

## (3) For one row

In [34]:
df.loc[0].isna()

A    False
B    False
C    False
D    False
Name: 0, dtype: bool

In [35]:
df.loc[1].isnull()

A    False
B     True
C    False
D    False
Name: 1, dtype: bool

## (4) For a slice

In [36]:
df.iloc[1:4, [1, 2, 3]].isnull()

,B,C,D
1,True,False,False
2,False,True,False
3,False,False,True


# 4. Statistics of Missing Values

## (1) Count of missing values in a column or row

We may count missing values in a column or row. If it contains too many missing values, we may exclude it from further analysis or data modelling. 

To count, we apply to isnull() another function sum(axis), where axis=0 (**default**) is for column and 1 for row.

In [37]:
df.isnull().sum(axis=0)

A    0
B    1
C    1
D    1
dtype: int64

In [38]:
df.isnull().sum(axis=1)

0    0
1    1
2    1
3    1
dtype: int64

To count the total missing values in a DataFrame object:

In [39]:
df.isnull().sum().sum()

3

## (2) Missing rate

Missing rate of each column (or row): count of missing values in the column (or row) / number of rows

In [40]:
df.isnull().sum(axis=0)/df.shape[0]

A    0.00
B    0.25
C    0.25
D    0.25
dtype: float64

Or, missing rate of each column in this way:

In [41]:
df.isnull().mean()

A    0.00
B    0.25
C    0.25
D    0.25
dtype: float64

In [42]:
df.isnull().sum(axis=1)/df.shape[1]

0    0.00
1    0.25
2    0.25
3    0.25
dtype: float64

**Sparsity** of the data: ratio of missing values in total:

In [43]:
df.isnull().sum().sum() / (df.shape[0]*df.shape[1])

0.1875

# 5. Filtering Missing Values

To get rows with missing value(s), we use .loc[] with .isnull() and .any().

## (1) Function any()

A DataFrame's **any()** function returns whether there exists any element being True, potentially over an axis. It returns False only if all elements are False or missing; otherwise, it returns True (at least one element is True or non-empty).

In [44]:
pd.Series([False, False]).any()

False

In [45]:
pd.Series([True, False]).any()

True

In [46]:
pd.Series([], dtype="float64").any()

False

In [47]:
pd.Series([np.nan]).any()

False

In [48]:
pd.Series(['ruc', np.nan]).any()

True

## (2) Check weather a row or column has missing values

We combine .isnull() and .any().

Below, the first row is filtered as in df.isnull() below it contains False values on all the 4 columns.

In [49]:
df.isnull()

,A,B,C,D
0,False,False,False,False
1,False,True,False,False
2,False,False,True,False
3,False,False,False,True


Below, any() or any(0) means to check the data vertically, i.e., on each of the columns.

In [50]:
df.isnull().any()

A    False
B     True
C     True
D     True
dtype: bool

Below, any(1) means to check the data horitontally, i.e., on each of the rows.

In [51]:
df.isnull().any(1)

0    False
1     True
2     True
3     True
dtype: bool

## (3) Row/Column retrieval based on missing values

Now we're ready to put all together.

Below, the first row of the data is filtered as in df.isnull() above it contains False values on all the 4 columns.

In [52]:
df

,A,B,C,D
0,a1,b1,1.0,5
1,a2,None,2.0,6
2,a3,b3,NaN,9
3,a4,b4,4.0,NaT


In [53]:
df.loc[df.isnull().any(1)]

,A,B,C,D
1,a2,None,2.0,6
2,a3,b3,NaN,9
3,a4,b4,4.0,NaT


We can return those rows without any missing value by the negation operation **~**:

In [54]:
df.loc[~(df.isnull().any(1))]

,A,B,C,D
0,a1,b1,1.0,5


To return those columns that have missing values:

In [55]:
df.loc[:,df.isnull().any()]

,B,C,D
0,b1,1.0,5
1,None,2.0,6
2,b3,NaN,9
3,b4,4.0,NaT


The following example shows how to return the rows or columns with all values missing. For that purpose, we use the function **all()**, which returns True is *all* values (in a row or column) are True. 

In [56]:
df2 = pd.DataFrame([['a1', 'bb', 'cc', np.NaN],
                    ['a2', 'bb', 'cc', np.NaN],
                    ['a3', 'bb', 'cc', np.NaN],
                    ['a4', 'bb', 'cc', np.NaN],
                    [np.NaN, np.NaN, np.NaN]],
                  columns=['A', 'B', 'C', 'D'])
df2

,A,B,C,D
0,a1,bb,cc,NaN
1,a2,bb,cc,NaN
2,a3,bb,cc,NaN
3,a4,bb,cc,NaN
4,NaN,NaN,NaN,NaN


In [57]:
df2.isnull()

,A,B,C,D
0,False,False,False,True
1,False,False,False,True
2,False,False,False,True
3,False,False,False,True
4,True,True,True,True


In [58]:
df2.loc[df2.isnull().all(1)]

,A,B,C,D
4,NaN,NaN,NaN,NaN


To return those columns where all values are missing:

In [59]:
df2.loc[:,df2.isnull().all(0)]

,D
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


# 6. Removing Missing Values

It's not necessary to remove all missing values. Whether or not to do so also depends on your tolerance. Missing values are often unavoidable in real datasets. Sometimes they do disclose some information.

## (1) To delete all

Function **dropna()** has a parameter *inplace*, which is False by default. This means a new DataFrame object iscreated with all missing values being removed.

In [60]:
df.dropna()

,A,B,C,D
0,a1,b1,1.0,5


In [61]:
df

,A,B,C,D
0,a1,b1,1.0,5
1,a2,None,2.0,6
2,a3,b3,NaN,9
3,a4,b4,4.0,NaT


## (2) To delete rows with missing values

In [62]:
df.dropna(axis=0)

,A,B,C,D
0,a1,b1,1.0,5


To delete rows with missing values on particular columns. In this exmaple, column A contains no missing value:

In [63]:
df.dropna(subset=['A'])

,A,B,C,D
0,a1,b1,1.0,5
1,a2,None,2.0,6
2,a3,b3,NaN,9
3,a4,b4,4.0,NaT


In this exmaple, column B contains a missing value:

In [64]:
df.dropna(subset=['B'])

,A,B,C,D
0,a1,b1,1.0,5
2,a3,b3,NaN,9
3,a4,b4,4.0,NaT


In this exmaple, column D contains a missing value:

In [65]:
df.dropna(subset=['D'])

,A,B,C,D
0,a1,b1,1.0,5
1,a2,None,2.0,6
2,a3,b3,NaN,9


## (3) To delete columns with missing values

In [66]:
df.dropna(axis=1)

,A
0,a1
1,a2
2,a3
3,a4


## (4) To delete rows or columns according to some conditions

For example, to keep columns whose missing rate is smaller than 0.1: 

In [67]:
df.loc[:,df.isnull().mean(axis=0) < 0.1]

,A
0,a1
1,a2
2,a3
3,a4


To keep rows whose missing rate is smaller than 0.1, i.e., to delete those larger than 0.1:

In [68]:
df.loc[df.isnull().mean(axis=1) < 0.1]

,A,B,C,D
0,a1,b1,1.0,5


We may also drop a row/column whose values are *all* missing.

In [69]:
df2

,A,B,C,D
0,a1,bb,cc,NaN
1,a2,bb,cc,NaN
2,a3,bb,cc,NaN
3,a4,bb,cc,NaN
4,NaN,NaN,NaN,NaN


In [70]:
df2.dropna(axis=0, how = 'all')

,A,B,C,D
0,a1,bb,cc,NaN
1,a2,bb,cc,NaN
2,a3,bb,cc,NaN
3,a4,bb,cc,NaN


In [71]:
df2.dropna(axis=1, how = 'all')

,A,B,C
0,a1,bb,cc
1,a2,bb,cc
2,a3,bb,cc
3,a4,bb,cc
4,NaN,NaN,NaN


In [72]:
df2.dropna(axis=0, how = 'all').dropna(axis=1, how = 'all')

,A,B,C
0,a1,bb,cc
1,a2,bb,cc
2,a3,bb,cc
3,a4,bb,cc


Compare the above to the following. What causes the difference?

In [73]:
df2.dropna(axis=1).dropna(axis=0)

""
0
1
2
3
4


In [74]:
df2.dropna(axis=0).dropna(axis=1)

,A,B,C,D


# 7. Filling in Missing Values

We use a DataFrame's function **fillna()** to fill in missing values. To make the new values really replace the missing values, we use inplace=True.

We can use one value for all missing values:

In [75]:
df.fillna(0)

,A,B,C,D
0,a1,b1,1.0,5
1,a2,0,2.0,6
2,a3,b3,0.0,9
3,a4,b4,4.0,0


In [76]:
df

,A,B,C,D
0,a1,b1,1.0,5
1,a2,None,2.0,6
2,a3,b3,NaN,9
3,a4,b4,4.0,NaT


We can use one value for all missing values of a particular column. In this example, column D's missing values will be replaced by -999.

In [77]:
df.D.fillna(-999, inplace=True)

df

,A,B,C,D
0,a1,b1,1.0,5
1,a2,None,2.0,6
2,a3,b3,NaN,9
3,a4,b4,4.0,-999


Function fillna(.)'s parameter **method**：
* forward padding/filling: pad/ffill
* backward padding/filling: bfill/backfill

df.fillna(methond='ffill') can be simplified as df.ffill(). This allows us to fill in using concrete values in adjacant cells in the DataFrame object. Note that missing values won't be passed in padding.

The following uses forward filling, starting from the first row and moves downwards.

In [78]:
df.ffill()

,A,B,C,D
0,a1,b1,1.0,5
1,a2,b1,2.0,6
2,a3,b3,2.0,9
3,a4,b4,4.0,-999


In [79]:
df

,A,B,C,D
0,a1,b1,1.0,5
1,a2,None,2.0,6
2,a3,b3,NaN,9
3,a4,b4,4.0,-999


The following uses backward filling, starting from the last colun and moves to the right.

In [80]:
df.bfill(axis=1)

,A,B,C,D
0,a1,b1,1.0,5
1,a2,2.0,2.0,6
2,a3,b3,9,9
3,a4,b4,4.0,-999


We may also use the mean of a column as the new value in fill in:

In [81]:
df.D.fillna(df.D.mean())

0      5
1      6
2      9
3   -999
Name: D, dtype: int64

Missing values won't be passed forward or backforward.

In [82]:
df3 = pd.DataFrame([[np.NaN, 'bb', 'cc', np.NaN],
                    ['a2', np.NaN, 'cc', np.NaN],
                    ['a3', 'bb', np.NaN, np.NaN],
                    ['a4', 'bb', 'cc', np.NaN],
                    [np.NaN, np.NaN, np.NaN]],
                  columns=['A', 'B', 'C', 'D'])
df3

,A,B,C,D
0,NaN,bb,cc,NaN
1,a2,NaN,cc,NaN
2,a3,bb,NaN,NaN
3,a4,bb,cc,NaN
4,NaN,NaN,NaN,NaN


In [83]:
df3.ffill()

,A,B,C,D
0,NaN,bb,cc,NaN
1,a2,bb,cc,NaN
2,a3,bb,cc,NaN
3,a4,bb,cc,NaN
4,a4,bb,cc,NaN


In [84]:
df3.bfill()

,A,B,C,D
0,a2,bb,cc,NaN
1,a2,bb,cc,NaN
2,a3,bb,cc,NaN
3,a4,bb,cc,NaN
4,NaN,NaN,NaN,NaN


# 8. Missing Values in Calculations

## (1) sum

sum() discards missing values.

In [85]:
df

,A,B,C,D
0,a1,b1,1.0,5
1,a2,None,2.0,6
2,a3,b3,NaN,9
3,a4,b4,4.0,-999


Sum for each column:

In [86]:
df.sum()

<ipython-input-86-7e5fdb616c56>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.sum()


A    a1a2a3a4
C         7.0
D        -979
dtype: object

Above, pay attention to the result of A: all strings are concatenated as the sum. Whereas B is excluded as there is a missing value so the remaining strings cannot be concatenated. For other numeric columns, missing values are simply ignored.

Sum for each row also discards missing values. When a row has a value, strings are discared as well.

In [87]:
df.sum(axis=1)

<ipython-input-87-f6cc363c055e>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.sum(axis=1)


0      6.0
1      8.0
2      9.0
3   -995.0
dtype: float64

## (2) counting

Missing values are excluded from counting.

Counting for columns:

In [88]:
df.count()

A    4
B    3
C    3
D    4
dtype: int64

Counting unique values for columns:

In [89]:
df.nunique()

A    4
B    3
C    3
D    4
dtype: int64

## (3) Aggregation and grouping

By default, aggregation discards missing values. 

In [90]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'], 
        'year': [2000, 2001, 2002, 2000, 2001, 2002],
        'pop': [np.NaN, 1.7, 3.6, 2.4, 2.9, 3.2],
        'debt': [2, 3, 4, 2, np.NaN, 6]
       }
dfs = pd.DataFrame(data)
dfs

,state,year,pop,debt
0,Ohio,2000,NaN,2.0
1,Ohio,2001,1.7,3.0
2,Ohio,2002,3.6,4.0
3,Nevada,2000,2.4,2.0
4,Nevada,2001,2.9,NaN
5,Nevada,2002,3.2,6.0


In [91]:
dfs.groupby(['year']).mean()

,pop,debt
year,,
2000,2.4,2.0
2001,2.3,3.0
2002,3.4,5.0


In [92]:
dfs.groupby('state')['pop'].max()

state
Nevada    3.2
Ohio      3.6
Name: pop, dtype: float64